In [1]:
import numpy as np
import statsmodels.api as sm

In [5]:
# The longley dataset is a time series dataset

data = sm.datasets.longley.load()
data.exog = sm.add_constant(data.exog)
data.exog.head()

,const,GNPDEFL,GNP,UNEMP,ARMED,POP,YEAR
0,1.0,83.0,234289.0,2356.0,1590.0,107608.0,1947.0
1,1.0,88.5,259426.0,2325.0,1456.0,108632.0,1948.0
2,1.0,88.2,258054.0,3682.0,1616.0,109773.0,1949.0
3,1.0,89.5,284599.0,3351.0,1650.0,110929.0,1950.0
4,1.0,96.2,328975.0,2099.0,3099.0,112075.0,1951.0


In [9]:
# Lets assume that the data is heteroskedastic and that we know the nature of the heteroskedasticity. We can then define sigma and use it to give us a GLD model


ols_resid = sm.OLS(data.endog, data.exog).fit().resid

In [10]:
old_resid

0     267.340030
1     -94.013942
2      46.287168
3    -410.114622
4     309.714591
5    -249.311215
6    -164.048956
7     -13.180357
8      14.304773
9     455.394095
10    -17.268927
11    -39.055043
12   -155.549974
13    -85.671308
14    341.931514
15   -206.757825
dtype: float64

In [11]:
resid_fit = sm.OLS(
    np.asarray(ols_resid)[1:], sm.add_constant(np.asarray(ols_resid)[:-1])
).fit()
print(resid_fit.tvalues[1])
print(resid_fit.pvalues[1])


-1.4390229839654487
0.17378444789040948


In [13]:
from scipy.linalg import toeplitz

In [14]:
toeplitz(range(5))

array([[0, 1, 2, 3, 4],
       [1, 0, 1, 2, 3],
       [2, 1, 0, 1, 2],
       [3, 2, 1, 0, 1],
       [4, 3, 2, 1, 0]])

In [15]:
order = toeplitz(range(len(ols_resid)))

In [16]:
# so that our error covariance structure is actually rho**order which defines and autocorrelation structure


sigma = rho ** order
gls_model = sm.GLS(data.endog, data.exog, sigma=sigma)
gls_results = gls_model.fit()

NameError: name 'rho' is not defined

In [17]:
glsar_model = sm.GLSAR(data.endog, data.exog, 1)
glsar_results = glsar_model.iterative_fit(1)
print(glsar_results.summary())


                           GLSAR Regression Results                           
Dep. Variable:                 TOTEMP   R-squared:                       0.996
Model:                          GLSAR   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     295.2
Date:                Thu, 26 Oct 2023   Prob (F-statistic):           6.09e-09
Time:                        13:28:14   Log-Likelihood:                -102.04
No. Observations:                  15   AIC:                             218.1
Df Residuals:                       8   BIC:                             223.0
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.468e+06   8.72e+05     -3.979      0.0

/home/susearc/Macos/py3.11/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
